# Table of Content

- [Table of Content](#table-of-content)
- [Data Inspection and Cleaning](#1-data-inspection-and-cleaning)
- [Covariate Imbalance Check](#2-check-covariate-imbalance-between-groups-pre-weighting)
- [Multinomial Logistic Regression](#3-estimate-propensity-scores-multinomial-logistic-regression)
- [Inverse Probability of Treatment Weights](#4-calculate-inverse-probability-of-treatment-weights-iptw)
- [Covariates Balance after Weighting](#5-check-covariate-balance-after-weighting)
- [Outcome Analysis](#6-weighted-outcome-analysis-using-iptw)
- [Results Output](#7-results-output)
  - [Comparison to Unadjusted Results](#71-comparison-of-unadjusted-table-2-and-iptw-adjusted-results)
  - [Supplementary Table Results](#72-supplementary-table-results)
  - [Results in Text](#73-results-text-changes)
  - [Method Section](#74-statistical-analysis-methods-section-changes)

# 0-General
[Back to Table of Content](#table-of-content)
# IPTW-Based Adjustment in a Pragmatic Cohort Study of Early Supportive Care in NSCLC

## Introduction
[Back to Table of Content](#table-of-content)

The ACCEPT® study investigates the impact of early, multimodal, patient-centered supportive care in individuals newly diagnosed with non-small-cell lung cancer (NSCLC). In line with international guidelines advocating for integrative psychosocial and palliative interventions, the study examines the effects of a flexible modular program combining health education, psycho-oncological counseling, and home-based artistic therapy on quality of life and psychological well-being. Given the complex symptom burden in NSCLC and the psychoneuroimmunological evidence supporting mind-body interventions, the aim was to evaluate whether an individualized early supportive care concept can provide measurable benefits alongside standard oncological care.

Due to ethical and practical limitations in randomizing patients in a real-world integrative oncology setting, a pragmatic cohort design was employed. Patients self-selected into one of three groups (multimodal, bimodal, or standard of care), based on individual preference and feasibility. While this design reflects clinical reality and enhances ecological validity, it also introduces the potential for confounding by indication, as treatment assignment was not randomized.

## Objective
[Back to Table of Content](#table-of-content)

The primary goal of this notebook is to estimate unbiased treatment effects of the ACCEPT® program by accounting for observed baseline differences between treatment groups. Specifically, we apply Inverse Probability of Treatment Weighting (IPTW) to adjust for confounding and approximate a pseudo-randomized comparison between the intervention and control groups. This approach enables us to examine treatment effects on quality of life (Trial Outcome Index, FACT-L) and other patient-reported outcomes while preserving the real-world context of patient preference.

## Methodological Role
[Back to Table of Content](#table-of-content)

To address the non-randomized group allocation and mitigate selection bias, propensity scores were estimated using a multinomial logistic regression model based on demographic and clinical baseline covariates (e.g., age, sex, ECOG status, tumor stage, and baseline PROM scores). Stabilized IPTW weights were calculated and applied to the analytical models to generate balanced pseudo-populations. The IPTW-adjusted models serve as the primary analytic strategy for comparing PROM changes across the treatment groups while preserving interpretability and robustness.

## Analysis steps
[Back to Table of Content](#table-of-content)

1. **Data Preparation and Cleaning:** Derive relevant baseline variables and recode variables.
2. **Descriptive Analysis:** Check for variable imbalance.
3. **Propensity Score Modeling:** Fit a multinomial logistic regression to estimate treatment probabilities.
4. **IPTW Construction:** Derive stabilized weights and assess balance pre- and post-weighting via standardized mean differences.
5. **Weighted Outcome Analysis:** Fit IPTW-adjusted linear regression models for PROM change scores. 

## Acknowledgements
[Back to Table of Content](#table-of-content)

Statistical analysis was conducted by Dr. Steven Schepanski, who also oversaw this notebook.

# Packages
[Back to Table of Content](#table-of-content)

In [53]:
# Define required packages
packages <- c("readxl", "dplyr", "tidyr", "ggplot2", "tableone", "MatchIt", "survey", "cobalt", "nnet")

# Function to install and load packages silently
install_and_load <- function(pkg) {
  if (!requireNamespace(pkg, quietly = TRUE)) install.packages(pkg, quiet = TRUE)
  suppressPackageStartupMessages(library(pkg, character.only = TRUE, quietly = TRUE, warn.conflicts = FALSE))
}

# Apply function to all packages
invisible(lapply(packages, install_and_load))

In [28]:
# Define the path and file
path <- "/Users/stevenschepanski/Documents/04_ANALYSIS/ACCEPT_IPTW/data/"
file <- "ACCEPT_IPTW.xlsx"

In [29]:
# Read the Excel file into a dataframe called "accept"
accept <- read_excel(paste0(path, file))

# 1-Data Inspection and Cleaning
[Back to Table of Content](#table-of-content)

In [30]:
# Inspect the structure of the data
str(accept)

# View first few rows of the data
head(accept)

tibble [116 x 16] (S3: tbl_df/tbl/data.frame)
 $ VPN                : num [1:116] 1 3 4 5 6 7 8 9 13 15 ...
 $ group              : chr [1:116] "bi" "bi" "bi" "bi" ...
 $ gender             : chr [1:116] "w" "m" "m" "w" ...
 $ age                : num [1:116] 67 66 70 61 66 80 66 56 72 71 ...
 $ ECOG > 1           : chr [1:116] "no" "no" "no" "yes" ...
 $ cancer stage       : num [1:116] 4 4 3 4 4 2 4 4 2 4 ...
 $ treatment intention: chr [1:116] "apal" "apal" "curativ" "apal" ...
 $ mod1               : chr [1:116] "8" "0" "0" "9" ...
 $ mod2               : num [1:116] 3 3 3 3 3 3 3 3 3 4 ...
 $ mod3               : chr [1:116] "0" "0" "0" "0" ...
 $ TOI_1              : chr [1:116] "40" "56" "41" "41" ...
 $ TOI_3              : chr [1:116] "54" "59" "27" "37" ...
 $ dTOI               : chr [1:116] "14" "3" "-14" "-4" ...
 $ HADS_1             : chr [1:116] "21" "14" "5" "23.692307692307693" ...
 $ HADS_3             : chr [1:116] "12" "7" "8" "16" ...
 $ dHADS              : chr [

VPN,group,gender,age,ECOG > 1,cancer stage,treatment intention,mod1,mod2,mod3,TOI_1,TOI_3,dTOI,HADS_1,HADS_3,dHADS
<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,bi,w,67,no,4,apal,8,3,0,40,54,14,21,12,-9
3,bi,m,66,no,4,apal,0,3,0,56,59,3,14,7,-7
4,bi,m,70,no,3,curativ,0,3,0,41,27,-14,5,8,3
5,bi,w,61,yes,4,apal,9,3,0,41,37,-4,23.692307692307693,16,-7.6923076923076934
6,multi,w,66,no,4,apal,12,3,6,48,70,22,13,2,-11
7,bi,m,80,no,2,curativ,7,3,0,52,58,6,11,4,-7


We rename complex column names and ensure that all variables are assigned the correct data type.
- `group`, `gender`, `ecog`, `stage`, and `intention` are converted to factors
- PROM scores and module variables are coerced to numeric

In [31]:
# Recode and clean variables, handling "NA" strings correctly
accept <- accept %>%
  rename(
    ecog = `ECOG > 1`,
    stage = `cancer stage`,
    intention = `treatment intention`
  ) %>%
  mutate(
    # Convert group to factor
    group = factor(group, levels = c("SOC", "bi", "multi")),
    
    # Recode gender and ECOG
    gender = factor(gender, levels = c("m", "w"), labels = c("male", "female")),
    ecog = factor(ecog, levels = c("no", "yes"), labels = c("0-1", ">1")),

    # Convert stage and intention to factors
    stage = factor(stage),
    intention = factor(intention),

    # Clean and convert module variables
    mod1 = as.numeric(na_if(mod1, "NA")),
    # mod2 = as.numeric(na_if(mod2, "NA")), ## this is already numeric, no need to convert
    mod3 = as.numeric(na_if(mod3, "NA")),

    # Clean and convert PROM variables
    TOI_1 = as.numeric(na_if(TOI_1, "NA")),
    TOI_3 = as.numeric(na_if(TOI_3, "NA")),
    dTOI  = as.numeric(na_if(dTOI, "NA")),
    HADS_1 = as.numeric(na_if(HADS_1, "NA")),
    HADS_3 = as.numeric(na_if(HADS_3, "NA")),
    dHADS  = as.numeric(na_if(dHADS, "NA"))
  )

In [32]:
# Check the structure of the cleaned data
str(accept)

tibble [116 x 16] (S3: tbl_df/tbl/data.frame)
 $ VPN      : num [1:116] 1 3 4 5 6 7 8 9 13 15 ...
 $ group    : Factor w/ 3 levels "SOC","bi","multi": 2 2 2 2 3 2 2 3 2 3 ...
 $ gender   : Factor w/ 2 levels "male","female": 2 1 1 2 2 1 2 2 2 1 ...
 $ age      : num [1:116] 67 66 70 61 66 80 66 56 72 71 ...
 $ ecog     : Factor w/ 2 levels "0-1",">1": 1 1 1 2 1 1 1 1 1 1 ...
 $ stage    : Factor w/ 3 levels "2","3","4": 3 3 2 3 3 1 3 3 1 3 ...
 $ intention: Factor w/ 2 levels "apal","curativ": 1 1 2 1 1 2 1 1 2 1 ...
 $ mod1     : num [1:116] 8 0 0 9 12 7 10 7 3 12 ...
 $ mod2     : num [1:116] 3 3 3 3 3 3 3 3 3 4 ...
 $ mod3     : num [1:116] 0 0 0 0 6 0 0 0 0 4 ...
 $ TOI_1    : num [1:116] 40 56 41 41 48 52 16 39 77 73 ...
 $ TOI_3    : num [1:116] 54 59 27 37 70 58 16 68 76 75 ...
 $ dTOI     : num [1:116] 14 3 -14 -4 22 6 0 29 -1 2 ...
 $ HADS_1   : num [1:116] 21 14 5 23.7 13 ...
 $ HADS_3   : num [1:116] 12 7 8 16 2 4 29 10 0 2 ...
 $ dHADS    : num [1:116] -9 -7 3 -7.69 -11 ...

# 2-Check Covariate Imbalance Between Groups (Pre-weighting)
[Back to Table of Content](#table-of-content)

In [35]:
# Define covariates to check
covariates <- c("TOI_1", "HADS_1", "gender", "ecog")

# Create TableOne stratified by group
table1 <- CreateTableOne(vars = covariates, strata = "group", data = accept, factorVars = c("gender", "ecog"))

# Print with standardized mean differences
print(table1, showAllLevels = TRUE, smd = TRUE)

                    Stratified by group
                     level  SOC           bi            multi         p     
  n                            43            38            35               
  TOI_1 (mean (SD))         54.61 (16.28) 50.43 (13.44) 52.24 (14.99)  0.479
  HADS_1 (mean (SD))        11.39 (7.03)  15.92 (8.12)  14.09 (8.46)   0.039
  gender (%)         male      25 (58.1)     16 (42.1)     10 (28.6)   0.031
                     female    18 (41.9)     22 (57.9)     25 (71.4)        
  ecog (%)           0-1       32 (78.0)     36 (97.3)     33 (94.3)   0.012
                     >1         9 (22.0)      1 ( 2.7)      2 ( 5.7)        
                    Stratified by group
                     test SMD   
  n                             
  TOI_1 (mean (SD))        0.186
  HADS_1 (mean (SD))       0.388
  gender (%)               0.412
                                
  ecog (%)                 0.416
                                


We assessed baseline comparability between the three treatment groups (SOC, bimodal, and multimodal) using standardized mean differences (SMDs) for key covariates: baseline TOI and HADS scores, gender, and ECOG performance status. SMDs > 0.1 indicate relevant imbalance.

- **TOI_1 (Trial Outcome Index)**: SMD = 0.186 → This indicates moderate imbalance between the groups, suggesting some differences in physical well-being at baseline.

- **HADS_1 (Hospital Anxiety and Depression Scale)**: SMD = 0.388 → Substantial imbalance, implying that psychological distress levels varied notably across groups prior to the intervention.

- **Gender**: SMD = 0.412 → Substantial imbalance, with more female patients in the intervention groups and more male patients in the SOC group.

- **ECOG performance status**: SMD = 0.416 → Substantial imbalance, indicating that patients with better physical performance (ECOG 0–1) were overrepresented in the intervention arms.

Taken together, these findings point to considerable differences in baseline characteristics across the three groups. Since group allocation was based on patient preference rather than randomization, these imbalances reflect potential **self-selection bias**. To reduce the risk of confounding in treatment effect estimation, we proceed with **propensity score weighting** using a multinomial logistic regression approach.

# 3-Estimate Propensity Scores (Multinomial Logistic Regression)
[Back to Table of Content](#table-of-content)

In [38]:
# Fit multinomial logistic regression model for treatment group
ps_model <- multinom(group ~ TOI_1 + HADS_1 + gender + ecog, data = accept, na.action = na.exclude)

# Extract predicted probabilities (one column per group)
ps <- predict(ps_model, type = "probs")

# Add PS columns to the dataset
accept <- accept %>%
  mutate(
    ps_soc   = ps[, "SOC"],
    ps_bi    = ps[, "bi"],
    ps_multi = ps[, "multi"]
  )

# weights:  18 (10 variable)
initial  value 116.452903 
iter  10 value 103.798072
final  value 103.715493 
converged


In [39]:
# Summarize the range of PS values
summary(accept[, c("ps_soc", "ps_bi", "ps_multi")])

     ps_soc            ps_bi           ps_multi      
 Min.   :0.04116   Min.   :0.0224   Min.   :0.05457  
 1st Qu.:0.18856   1st Qu.:0.2401   1st Qu.:0.23426  
 Median :0.35832   Median :0.3237   Median :0.34825  
 Mean   :0.35849   Mean   :0.3208   Mean   :0.32075  
 3rd Qu.:0.47888   3rd Qu.:0.3958   3rd Qu.:0.42809  
 Max.   :0.92303   Max.   :0.7120   Max.   :0.46048  
 NA's   :10        NA's   :10       NA's   :10       

We fitted a multinomial logistic regression model to estimate each patient's probability of receiving their observed treatment assignment, conditional on baseline characteristics (TOI_1, HADS_1, gender, ecog). The predicted probabilities serve as group-specific propensity scores, which will be used for inverse probability weighting.

These PS values represent each patient's estimated probability of receiving their observed treatment, given their baseline characteristics. They allow us to reweight the sample, so that it mimics a randomized trial — by creating a pseudo-population where covariates are balanced across groups.

- No extreme probabilities (0 or 1 ) → This means we avoid infinite weights.
- Most values are in the range ~0.05 to 0.9 → Suggests reasonable overlap between treatment groups.
- No convergence issues → The model converged normally.

In [40]:
# Identify participants with missing propensity scores
accept %>% filter(is.na(ps_soc) | is.na(ps_bi) | is.na(ps_multi))

VPN,group,gender,age,ecog,stage,intention,mod1,mod2,mod3,TOI_1,TOI_3,dTOI,HADS_1,HADS_3,dHADS,ps_soc,ps_bi,ps_multi
<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
33,bi,female,57,0-1,3,curativ,3,3,1,NA,36,NA,28,22,-6,NA,NA,NA
49,multi,female,81,0-1,4,apal,9,3,4,NA,72,NA,14,9,-5,NA,NA,NA
100,bi,male,59,0-1,3,curativ,0,3,0,NA,62,NA,11,14,3,NA,NA,NA
142,SOC,male,81,0-1,2,curativ,0,0,0,NA,36,NA,24,20,-4,NA,NA,NA
149,SOC,female,71,0-1,4,apal,0,0,0,NA,49,NA,11,11,0,NA,NA,NA
158,SOC,female,66,0-1,3,curativ,0,0,0,51,53,2,NA,17,NA,NA,NA,NA
178,SOC,female,58,NA,3,curativ,0,0,0,44,40,-4,25,24,-1,NA,NA,NA
201,bi,male,77,NA,3,curativ,3,3,0,56,41,-15,9,8,-1,NA,NA,NA
219,bi,male,67,0-1,3,curativ,4,3,0,NA,73,NA,NA,1,NA,NA,NA,NA


In [41]:
# Create new data frame excluding missing propensity scores
accept_iptw <- accept %>%
  filter(!is.na(ps_soc) & !is.na(ps_bi) & !is.na(ps_multi))

# Check the number of rows before and after
nrow(accept)       # original data
nrow(accept_iptw)  # after exclusion

[1] 116

[1] 106

We exclude patients with missing values in any of the estimated propensity score columns. The cleaned dataset `accept_iptw` will be used for IPTW estimation.

# 4-Calculate Inverse Probability of Treatment Weights (IPTW)
[Back to Table of Content](#table-of-content)

In [42]:
# Create unstabilized IPTW weights based on actual group assignment
accept_iptw <- accept_iptw %>%
  mutate(
    weight = case_when(
      group == "SOC"   ~ 1 / ps_soc,
      group == "bi"    ~ 1 / ps_bi,
      group == "multi" ~ 1 / ps_multi
    )
  )

# Summary of weights
summary(accept_iptw$weight)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.083   2.201   2.480   2.915   3.377  12.145 

In [46]:
# Inspect extreme values
round(quantile(accept_iptw$weight, probs = c(0, 0.01, 0.05, 0.5, 0.95, 0.99, 1)), 4)

0%      1%      5%     50%     95%     99%    100% 
 1.0834  1.0886  1.3579  2.4795  5.1064  5.9714 12.1449

Weights were calculated as the inverse of the estimated propensity score for each patient's observed treatment group. These weights are used to adjust for baseline imbalances and simulate random treatment allocation in a pseudo-population.

The distribution of the weights was as follows:
- **Minimum**: 1.08
- **Median**: 2.48
- **95th percentile**: 5.11
- **Maximum**: 12.14

Most weights fall within a reasonable range, suggesting stable estimation. A small number of higher weights (up to ~12) may introduce mild instability, but we consider them acceptable for initial analyses. Sensitivity analyses with trimmed or stabilized weights can be considered later if needed.

# 5-Check Covariate Balance After Weighting
[Back to Table of Content](#table-of-content)

In [47]:
# Define survey design object using IPTW
design_iptw <- svydesign(ids = ~1, data = accept_iptw, weights = ~weight)

# Define covariates
covariates <- c("TOI_1", "HADS_1", "gender", "ecog")

In [50]:
# Balance assessment: weighted vs unweighted
bal_iptw <- bal.tab(group ~ TOI_1 + HADS_1 + gender + ecog,
                    data = accept_iptw,
                    weights = accept_iptw$weight,
                    method = "weighting",
                    estimand = "ATE",
                    un = TRUE)

# Print summary of balance table
print(bal_iptw)

Balance summary across all treatment pairs
                 Type Max.Diff.Un Max.Diff.Adj
TOI_1         Contin.      0.3598       0.0406
HADS_1        Contin.      0.6741       0.1505
gender_female  Binary      0.3375       0.0219
ecog_>1        Binary      0.2074       0.0529

Effective sample sizes
            SOC    bi multi
Unadjusted 38.  34.   34.  
Adjusted   25.1 30.39 30.97


Balance assessment was performed using standardized mean differences (SMDs) before and after weighting. Prior to IPTW, substantial imbalances were observed, particularly for HADS_1 (SMD = 0.674) and gender (SMD = 0.338). After weighting, covariate balance improved markedly:

- TOI_1, gender, and ECOG achieved acceptable balance (SMD < 0.1)
- HADS_1 remained slightly imbalanced (SMD = 0.151), suggesting partial correction

Effective sample sizes were modestly reduced after weighting (SOC: 38 → 25.1), reflecting typical effects of weighting procedures.

Overall, IPTW substantially improved comparability between groups and supports proceeding to weighted outcome analysis. Caution should be exercised in interpreting results related to psychological outcomes due to residual imbalance in HADS_1.

# 6-Weighted Outcome Analysis Using IPTW
[Back to Table of Content](#table-of-content)

In [64]:
# Create weighted survey design object
design_iptw <- svydesign(ids = ~1, data = accept_iptw, weights = ~weight)

In [65]:
# Model 1: dTOI as outcome
model_dTOI <- svyglm(dTOI ~ group, design = design_iptw)
summary(model_dTOI)


Call:
svyglm(formula = dTOI ~ group, design = design_iptw)

Survey design:
svydesign(ids = ~1, data = accept_iptw, weights = ~weight)

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept)    0.632      2.909   0.217  0.82848   
groupbi        3.830      3.662   1.046  0.29813   
groupmulti    11.520      3.668   3.141  0.00221 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 173.8054)

Number of Fisher Scoring iterations: 2


Using IPTW-adjusted linear regression, we estimated differences in the change of the Trial Outcome Index (dTOI) across treatment groups. Participants in the multimodal group experienced a statistically significant improvement of 11.52 points in dTOI compared to the standard of care group (β = 11.52, SE = 3.67, p = 0.002), while the bimodal group showed a smaller, non-significant improvement by 3.83 points compared to SOC (β = 3.83, SE = 3.66, p = 0.30). These results suggest that the full ACCEPT® intervention may confer clinically meaningful benefits in physical and functional well-being.

The SOC group, serving as the reference, had a mean change in dTOI of approximately 0.63, which was not significantly different from 0, indicating no meaningful change over time without intervention.

In [66]:
# Model 2: dHADS as outcome
model_dHADS <- svyglm(dHADS ~ group, design = design_iptw)
summary(model_dHADS)


Call:
svyglm(formula = dHADS ~ group, design = design_iptw)

Survey design:
svydesign(ids = ~1, data = accept_iptw, weights = ~weight)

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   0.1589     1.1848   0.134 0.893573    
groupbi      -3.8571     1.5696  -2.457 0.015679 *  
groupmulti   -6.1831     1.6249  -3.805 0.000242 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 38.75772)

Number of Fisher Scoring iterations: 2


The IPTW-weighted model indicated that both intervention groups experienced significantly greater reductions in psychological distress (dHADS) compared to the standard of care (SOC). The bimodal group improved by an average of 3.86 points (β = 3.86, SE = 1.57, p = 0.016), and the multimodal group improved by 6.18 points (β = 6.18, SE = 1.62, p < 0.001). The SOC group showed no significant change (β = 0.16, p = 0.89). These findings indicate statistically relevant emotional benefits from both versions of the ACCEPT® program, with the full multimodal intervention yielding the strongest effects.

In [67]:
confint(model_dHADS)

,2.5 %,97.5 %
(Intercept),-2.191217,2.509034
groupbi,-6.970309,-0.743911
groupmulti,-9.406091,-2.960055


Based on the IPTW-adjusted regression model, both intervention groups showed clinically relevant improvements in psychological distress (dHADS) compared to the SOC group. The bimodal group improved by 3.86 points (95% CI [–6.97, –0.74]), a change that lies within the established MCID range for the HADS (1.7 to 5.7 points). Although the confidence interval includes values below the MCID threshold, the majority of the plausible effect range supports a meaningful clinical benefit.

The multimodal group improved by 6.18 points (95% CI [–9.41, –2.96]), clearly exceeding the MCID threshold in both its point estimate and confidence bounds. This indicates a robust and highly likely clinical benefit from the full intervention program.

The SOC group showed no meaningful change (β = 0.16, 95% CI [–2.19, 2.51]).

# 7-Results Output
[Back to Table of Content](#table-of-content)

## 7.1-Comparison of Unadjusted (Table 2) and IPTW-Adjusted Results
[Back to Table of Content](#table-of-content)

To evaluate the robustness of intervention effects, we compared unadjusted mean changes in PROMs (Table 2) with IPTW-adjusted estimates from weighted regression models. The unadjusted analysis suggested significant improvements in dTOI and dHADS for both the bimodal and multimodal groups compared to SOC.

After adjustment for baseline covariates (TOI_1, HADS_1, gender, and ECOG), the effects of the **multimodal intervention** remained statistically significant and of similar magnitude (e.g., dTOI: +12.91 unadjusted vs. +11.52 adjusted; dHADS: –6.26 vs. –6.18). In contrast, the **effect sizes for the bimodal group were attenuated** after weighting (e.g., dTOI: +4.97 vs. +3.83, no longer significant), suggesting some influence of self-selection bias.

These findings support the added benefit of the full multimodal intervention and highlight the importance of accounting for baseline differences in observational group allocation.

## 7.2-Supplementary Table Results
[Back to Table of Content](#table-of-content)

| Outcome | Comparison           | Estimate (β) | 95% CI                     | p-value |
|---------|----------------------|--------------|----------------------------|---------|
| dTOI    | Bimodal vs. SOC      | +3.83        | [–3.37, +11.03]            | 0.298   |
|         | Multimodal vs. SOC   | +11.52       | [+4.29, +18.75]            | 0.002   |
| dHADS   | Bimodal vs. SOC      | –3.86        | [–6.97, –0.74]             | 0.016   |
|         | Multimodal vs. SOC   | –6.18        | [–9.41, –2.96]             | <0.001  |

Estimates are based on inverse probability of treatment weighting (IPTW) using multinomial propensity scores. 
Models were adjusted for baseline TOI (TOI_1), baseline HADS (HADS_1), gender, and ECOG performance status. 
The SOC group served as the reference category. CI = confidence interval.

## 7.3-Results Text Changes
[Back to Table of Content](#table-of-content)

These patterns remained largely consistent after adjustment for baseline differences using propensity score weighting. In the IPTW-adjusted analysis, the multimodal group continued to show significant improvements across primary outcomes, including an average increase of 11.52 points in physical well-being (dTOI) and a 6.18-point reduction in psychological distress (dHADS), both statistically and clinically meaningful. The bimodal group also demonstrated a 3.86-point reduction in dHADS, falling within the expected range of a minimal clinically important difference, although its improvement in dTOI was no longer statistically significant (Supplementary Table X).

## 7.4-Statistical Analysis Methods Section Changes
[Back to Table of Content](#table-of-content)

To account for potential baseline differences due to the non-randomized allocation of participants, we applied inverse probability of treatment weighting (IPTW) using multinomial propensity scores. Propensity scores were estimated via multinomial logistic regression including baseline TOI, baseline HADS, gender, and ECOG performance status as covariates. Stabilized weights were not applied. Weighted linear regression models were then used to estimate group differences in primary outcomes (dTOI and dHADS). Covariate balance before and after weighting was assessed using standardized mean differences (SMDs), with values <0.1 considered indicative of adequate balance. The standard of care (SOC) group served as the reference category in all models.